In [1]:
!pip install ipython-sql
%load_ext sql
%sql sqlite://

In [2]:
%%sql
CREATE TABLE sales (
  "customer_id" VARCHAR(1),
  "order_date" DATE,
  "product_id" INTEGER
);

INSERT INTO sales
  ("customer_id", "order_date", "product_id")
VALUES
  ('A', '2021-01-01', '1'),
  ('A', '2021-01-01', '2'),
  ('A', '2021-01-07', '2'),
  ('A', '2021-01-10', '3'),
  ('A', '2021-01-11', '3'),
  ('A', '2021-01-11', '3'),
  ('B', '2021-01-01', '2'),
  ('B', '2021-01-02', '2'),
  ('B', '2021-01-04', '1'),
  ('B', '2021-01-11', '1'),
  ('B', '2021-01-16', '3'),
  ('B', '2021-02-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-01', '3'),
  ('C', '2021-01-07', '3');
 

CREATE TABLE menu (
  "product_id" INTEGER,
  "product_name" VARCHAR(5),
  "price" INTEGER
);

INSERT INTO menu
  ("product_id", "product_name", "price")
VALUES
  ('1', 'sushi', '10'),
  ('2', 'curry', '15'),
  ('3', 'ramen', '12');
  

CREATE TABLE members (
  "customer_id" VARCHAR(1),
  "join_date" DATE
);

INSERT INTO members
  ("customer_id", "join_date")
VALUES
  ('A', '2021-01-07'),
  ('B', '2021-01-09');

 * sqlite://
Done.
15 rows affected.
Done.
3 rows affected.
Done.
2 rows affected.


ResourceClosedError: This result object does not return rows. It has been closed automatically.

In [22]:
%%sql
-- 1. What is the total amount each customer spent at the restaurant?
select sales.customer_id,sum(menu.price) as total_amount
from sales 
inner join
menu  on sales.product_id = menu.product_id
group by sales.customer_id 
order by sales.customer_id ;

 * sqlite://
Done.


customer_id,total_amount
A,76
B,74
C,36


In [6]:
%%sql
-- 2. How many days has each customer visited the restaurant?
select customer_id,count(distinct order_date)
from sales
group by customer_id;

 * sqlite://
Done.


customer_id,count(distinct order_date)
A,4
B,6
C,2


In [8]:
%%sql
-- 3. What was the first item from the menu purchased by each customer?
select t1.customer_id,t2.product_name from
(select customer_id,order_date,product_id, row_number() over(partition by customer_id order by order_date) as row_num
from sales) t1
inner join 
menu t2 on t1.product_id = t2.product_id
where t1.row_num = 1;

 * sqlite://
Done.


customer_id,product_name
A,sushi
B,curry
C,ramen


In [9]:
%%sql
-- 4. What is the most purchased item on the menu and how many times was it purchased by all customers?
select t1.customer_id,t2.product_name,count(t1.product_id) from
sales t1
inner join
menu t2 on t1.product_id = t2.product_id 
where t1.product_id =
(select t.product_id from (
  select product_id,count(product_id) as total_count
from sales
group by customer_id,product_id
order by total_count Desc limit 1) t)
group by t1.customer_id,t2.product_name;


 * sqlite://
Done.


customer_id,product_name,count(t1.product_id)
A,ramen,3
B,ramen,2
C,ramen,3


In [10]:
%%sql
select t.customer_id,t.product_name from 
(select t1.customer_id,t2.product_name,count(t1.product_id),rank() over(partition by customer_id order by count(t1.product_id) desc) as row_num
from sales t1
inner join 
menu t2 on t1.product_id = t2.product_id
group by t1.customer_id,t2.product_name) t
where t.row_num =1; 

 * sqlite://
Done.


customer_id,product_name
A,ramen
B,sushi
B,ramen
B,curry
C,ramen


In [23]:
%%sql
-- 6. Which item was purchased first by the customer after they became a member?
select t.customer_id,t.product_name,t.order_date from
(select sales.customer_id,menu.product_name,sales.order_date,
row_number() over(partition by sales.customer_id order by sales.order_date) as row_num
from sales 
inner join 
menu on sales.product_id = menu.product_id
inner join 
members on sales.customer_id = members.customer_id
where sales.order_date >= members.join_date) t where t.row_num = 1
;

 * sqlite://
Done.


customer_id,product_name,order_date
A,curry,2021-01-07
B,sushi,2021-01-11


In [24]:
%%sql
-- 7. Which item was purchased just before the customer became a member?
select t.customer_id,t.product_name,t.order_date from
(select sales.customer_id,menu.product_name,sales.order_date,
row_number() over(partition by sales.customer_id order by sales.order_date desc) as row_num
from sales 
inner join 
menu on sales.product_id = menu.product_id
inner join 
members on sales.customer_id = members.customer_id
where sales.order_date < members.join_date) t where t.row_num = 1
;

 * sqlite://
Done.


customer_id,product_name,order_date
A,sushi,2021-01-01
B,sushi,2021-01-04


In [25]:
%%sql
-- 8. What is the total items and amount spent for each member before they became a member?
select sales.customer_id,count(menu.product_name),sum(price) as total_amount
from sales 
inner join 
menu on sales.product_id = menu.product_id
inner join 
members on sales.customer_id = members.customer_id
where sales.order_date < members.join_date
group by sales.customer_id;

 * sqlite://
Done.


customer_id,count(menu.product_name),total_amount
A,2,25
B,3,40


In [27]:
%%sql
-- 9.  If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?
select customer_id,
sum(case when product_name = 'sushi' then 20 * (menu.price) else (10 * menu.price) end) as points
from sales 
inner join
menu on sales.product_id = menu.product_id
group by customer_id
order by customer_id;

 * sqlite://
Done.


customer_id,points
A,860
B,940
C,360


In [20]:
%%sql
-- 10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?
select sales.customer_id,
sum(case when sales.order_date between members.join_date and date(members.join_date,'+7 days')  then (20*price) 
    when sales.order_date > date(members.join_date,'+7 days')   then (10*price) end) as points_earned
from
sales 
inner join
menu on sales.product_id = menu.product_id
inner join
members members on sales.customer_id = members.customer_id  
group by ddsal.customer_id;


 * sqlite://
Done.


customer_id,points_earned
A,1020
B,560
